In [3]:
import sqlite3

# 01 SQLite 사용

## 작업순서

Python       DBMS - DB<br>
Client ---> connection<br>
cursor --->  작업공간<br>
connection - cursor<br>

In [2]:
con = sqlite3.connect(':memory:')

NameError: name 'sqlite3' is not defined

In [8]:
cur = con.cursor()

In [12]:
dir(cur)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

### 01.01 Create Database, Insert Data

cur.execute() #SQL문 한개 실행함<br>
cur.excutemany() #SQL문 1개가 반복실행됨<br>
cur.executescript() #SQL문 여러개 한번에 실행(비표준)<br>

In [13]:
cur.execute('''
    CREATE TABLE CITY (
        CNO INTEGER PRIMARY KEY,
        CNAME TEXT
    )
''')

In [14]:
cur.execute('''
    INSERT INTO CITY (CNO, CNAME)
    VALUES(1, "LONDON");
''')

In [15]:
#error test
cur.execute('''
    INSERT INTO CITY (CNO, CNAME)
    VALUES("1", "LONDON");
''')

IntegrityError: UNIQUE constraint failed: CITY.CNO

In [16]:
cur.execute('''
    INSERT INTO CITY
    VALUES(2, "Paris");
''')

In [18]:
#Omit CNO
cur.execute('''
    INSERT INTO CITY (CNAME)
    VALUES("Rome");
''')

In [22]:
#PK NULL CASE
cur.execute('''
    INSERT INTO CITY
    VALUES(NULL, "Rome");
''')

### 01.02 Fetch
cur.fetchone() #Tuple/Record 1개 가져옴<br>
cur.fetchmany() #N개 가져옴<br>
cur.fetchall #전부가져옴<br>

In [37]:
cur.execute('SELECT * FROM CITY')

In [38]:
cur.fetchone()

(1, 'LONDON')

In [39]:
cur.fetchmany(2)

[(2, 'Paris'), (3, 'Rome')]

In [41]:
cur.execute('SELECT * FROM CITY')

In [42]:
cur.fetchall()

[(1, 'LONDON'), (2, 'Paris'), (3, 'Rome'), (4, 'Rome')]

In [43]:
cur.execute('''
    CREATE TABLE SUPPLIER(
        SNO INTEGER PRIMARY KEY,
        SNAME TEXT NOT NULL DEFAULT 'NONAME',
        CNO INTEGER NOT NULL
    )
''')

In [44]:
# qmark = ?, name = k:v
sname = 'Smith'
cno = 1

cur.execute('''
    INSERT INTO SUPPLIER VALUES(NULL, ?, ?)
''', (sname, cno))

In [45]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, 'Smith', 1)]

In [47]:
data = (('Johns', 2), ('Adams', 1), ('Blake', 3))

cur.executemany('''
    INSERT INTO SUPPLIER VALUES(NULL, ?, ?)
''', data)

In [48]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, 'Smith', 1), (2, 'Johns', 2), (3, 'Adams', 1), (4, 'Blake', 3)]

In [57]:
cur.execute('''
    CREATE TABLE PART(
        PNO INTEGER PRINMARY KEY AUTOINCRREMENT,
        PNAME TEXT
    )
''')

OperationalError: table PART already exists

In [58]:
cur.execute('''
    INSERT INTO PART
    VALUES(:pno, :pname)
''', {'pno':1, 'pname':'Screw', 'agew':'agsd'})

In [59]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (None, 'Nut'), (None, 'Bolt'), (None, 'Cam'), (1, 'Screw')]

In [60]:
data = [{'pname':'Nut'},{'pname':'Bolt'},{'pname':'Cam'}]

cur.executemany('''
    INSERT INTO PART
    VALUES(NULL, :pname)
''', data)

In [61]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'),
 (None, 'Nut'),
 (None, 'Bolt'),
 (None, 'Cam'),
 (1, 'Screw'),
 (None, 'Nut'),
 (None, 'Bolt'),
 (None, 'Cam')]

In [62]:
cur.execute('''
    CREATE TABLE SELLS(
        SNO INTEGER NOT NULL,
        PNO INTEGER NOT NULL,
        PRICE INTEGER NOT NULL
    );
''')

In [68]:
key1 = 'Screw'
key2 = 'Smith'

cur.execute('''
    SELECT PNO FROM PART
    WHERE PNAME=?
''', [key1])

rst = cur.fetchone()
if rst:
    pno = rst[0]
    
cur.execute('''
    SELECT SNO FROM SUPPLIER
    WHERE SNAME=?
''', [key2])

rst = cur.fetchone()
if rst:
    sno = rst[0]
    
pno,sno

(1, 1)

In [69]:
data = ['Screw', 'Smith', 10]

#LIMIT는 가져오는 결과값을 제한한다.

cur.execute('''
    INSERT INTO SELLS
    VALUES(
        (SELECT PNO FROM PART WHERE PNAME=? LIMIT 0, 1), 
        (SELECT SNO FROM SUPPLIER WHERE SNAME=? LIMIT 0, 1),
        ?
    )
''', data)

In [70]:
cur.execute('SELECT * FROM SELLS')
cur.fetchall()

[(1, 1, 10)]

In [74]:
CITY = dict()

cur.execute('SELECT * FROM CITY')
for row in cur.fetchall():
    CITY[row[1]] = row[0]

In [73]:
CITY['London']

KeyError: 'London'

In [75]:
data = [(1,2,8),
       (2,4,38),
       (3,1,11),
       ]

cur.executemany('INSERT INTO SELLS VALUES(?,?,?)', data)

In [1]:
con.close()

NameError: name 'con' is not defined

In [5]:
con = sqlite3.connect('0905.db')
cur = con.cursor()

In [6]:
cur.execute('''
    CREATE TABLE PART(
        PNO INTEGER PRIMARY KEY,
        PNAME TEXT
    )
''')

In [9]:
cur.executescript('''
        INSERT INTO PART VALUES(1, 'Screw');
        INSERT INTO PART(PNAME) VALUES('Cam');
        INSERT INTO PART(PNAME) VALUES('Bolt');
        INSERT INTO PART(PNAME) VALUES('Nut');
''')

In [11]:
cur.execute('INSERT INTO PART VALUES(NULL, "Part1")')

In [12]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Cam'), (3, 'Bolt'), (4, 'Nut'), (5, 'Part1')]

In [13]:
con.close()

In [14]:
con = sqlite3.connect('0905.db')
cur = con.cursor()

In [15]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Cam'), (3, 'Bolt'), (4, 'Nut')]

위 실행 구문에서는 executescript에서는 commit을 기본적으로 실행하는 것을 보여준다.
밑에 있는 cur.execute에서는 commit이 자동적으로 실행되지 않는다.

SQLite에서 작업한 데이터베이스는
DB Browser for SQLite을 설치해서 간단하게(No Installer) 볼 수 있다.

In [ ]:
con.commit() #DB에 반영시킴

In [18]:
cur.lastrowid

0

In [20]:
cur.execute('SELECT * FROM PART')
cur.fetchall()[-1]

(4, 'Nut')

In [22]:
con.close()

In [23]:
con = sqlite3.connect('0905.db')

In [24]:
cur = con.cursor()

In [29]:
cur.executescript('''
    DROP TABLE IF EXISTS TEMP;
    CREATE TABLE TEMP(
        PK INTEGER NOT NULL UNIQUE,
        NAME TEXT NOT NULL
    );
    
    INSERT INTO TEMP VALUES(1, 'TEST1');
    INSERT INTO TEMP VALUES(2, 'TEST2');
    INSERT INTO TEMP VALUES(2, 'TEST3');
''')

IntegrityError: UNIQUE constraint failed: TEMP.PK

In [31]:
#제약조건까지 그대로 싣는다.
for line in con.iterdump():
        print(line)

BEGIN TRANSACTION;
CREATE TABLE PART(
        PNO INTEGER PRIMARY KEY,
        PNAME TEXT
    );
INSERT INTO "PART" VALUES(1,'Screw');
INSERT INTO "PART" VALUES(2,'Cam');
INSERT INTO "PART" VALUES(3,'Bolt');
INSERT INTO "PART" VALUES(4,'Nut');
CREATE TABLE TEMP(
        PK INTEGER NOT NULL UNIQUE,
        NAME TEXT NOT NULL
    );
INSERT INTO "TEMP" VALUES(1,'TEST1');
INSERT INTO "TEMP" VALUES(2,'TEST2');
COMMIT;


# SQLite를 통한 예제 구현

In [63]:
con = sqlite3.connect('sns.db')
cur = con.cursor()

In [64]:
cur.executescript('''
    DROP TABLE IF EXISTS POST;
    DROP TABLE IF EXISTS HASHTAG;
    DROP TABLE IF EXISTS POSTTAG;
''')

In [65]:
cur.executescript('''
    CREATE TABLE POST(
        PK INTEGER PRIMARY KEY,
        CONTENT TEXT
    );
    CREATE TABLE HASHTAG(
        PK INTEGER PRIMARY KEY,
        COUNT INTEGER DEFAULT 0,
        NAME TEXT
    );
    CREATE TABLE POSTTAG(
        PPK INTEGER NOT NULL,
        HPK INTEGER NOT NULL
    );
''')

In [66]:
#HASHTAG Pool <- Tag가 있는지 검사
#있으면 PK, 없으면 새롭게 Insert
data1 = {'tag':'Python'}

cur.execute('''
    SELECT COUNT(*) FROM HASHTAG
    WHERE NAME=:tag
''',data1)

if cur.fetchone()[0] == 0:
    cur.execute('''
        INSERT INTO HASHTAG(NAME) VALUES(:tag)
    ''', data1)
    con.commit()

cur.execute('''
    SELECT PK
    FROM HASHTAG
    WHERE NAME=:tag
    LIMIT 0,1
''', data1)

PPK = cur.fetchone()[0]

#Posting 하는 부분
data2 = {'content': 'contentA', 'tags':['Python']}

cur.execute('INSERT INTO POST(CONTENT) VALUES(:content)', data2)
con.commit()
TPK = cur.lastrowid;

for tag in data['tags']:
    #위에다가 tag를 넣으면 PK를 리턴할 것임.
    cur.execute('INSERT INTO POSTTAG VALUES(?,?)',[PPK,TPK])
    cur.execute('''
        UPDATE HASHTAG SET COUNT = COUNT+1
            WHERE PK=?
    ''', [HPK])
    con.commit()

데이터베이스간의 관계보다, 데이터베이스의 처리에 집중하게 되었다는 점이 RDBMS의 가장 큰 단점이다. 그래서 객체-관계로 접근하는 것이 ORM이다.
PostgreSQL, SQLite등을 지원하는 SQLAlchemy를 쓴다.<br>
여기서 ORM은 Object와 Relation을 자동으로 Mapping해준다는 것

# 03 ORM 사용

In [67]:
#!pip list
#!pip install SQLAlchemy

Package                       Version
----------------------------- ---------------
aiobotocore                   2.4.2
aiofiles                      22.1.0
aiohttp                       3.8.3
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-catalogs             0.2.0
anaconda-client               1.12.0
anaconda-navigator            2.4.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcal

ORM은 core을 사용할 수 있게 감싸둔 것이고, Core의 기능은 다음과 같다.<br>
Engine : dialect / connecton pooling : 연결 수립, 데이터 전달 등등...<br>
...

In [71]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [72]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [74]:
meta = MetaData()

In [76]:
user = Table('USER', meta, 
            Column('PK', Integer, primary_key=True),
            Column('NAME', Text, nullable=True),
            )

In [77]:
meta.create_all(engine)
#여기서 나오는 것이 뒤어서 돌아가는 SQL문이다.(echo=True라서 반환됨)

2023-09-05 12:24:41,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-05 12:24:41,396 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-05 12:24:41,397 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:24:41,399 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-05 12:24:41,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:24:41,402 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-05 12:24:41,403 INFO sqlalchemy.engine.Engine [no key 0.00085s] ()
2023-09-05 12:24:41,404 INFO sqlalchemy.engine.Engine COMMIT


In [78]:
meta.clear()

In [79]:
meta.tables

FacadeDict({})

In [80]:
meta.create_all(engine)

2023-09-05 12:26:03,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-05 12:26:03,755 INFO sqlalchemy.engine.Engine COMMIT


In [81]:
meta.reflect(engine)

2023-09-05 12:27:05,873 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-09-05 12:27:05,875 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:05,876 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("USER")
2023-09-05 12:27:05,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:05,878 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-09-05 12:27:05,878 INFO sqlalchemy.engine.Engine [raw sql] ('USER',)
2023-09-05 12:27:05,880 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("USER")
2023-09-05 12:27:05,880 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:05,881 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("USER")
2023-09-05 12:27:05,881 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:05,882 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite

In [82]:
meta.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', INTEGER(), table=<USER>, primary_key=True, nullable=False), Column('NAME', TEXT(), table=<USER>), schema=None)})

In [83]:
type(user), dir(user)

(sqlalchemy.sql.schema.Table,
 ['__bool__',
  '__class__',
  '__class_getitem__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__getstate__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__invert__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__nonzero__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__sizeof__',
  '__slots__',
  '__str__',
  '__subclasshook__',
  '__visit_name__',
  '__weakref__',
  '_all_selected_columns',
  '_annotate',
  '_annotations',
  '_annotations_cache_key',
  '_anonymous_fromclause',
  '_assert_no_memoizations',
  '_autoincrement_column',
  '_autoload',
  '_cache_key_traversal',
  '_clone',
  '_cloned_set',
  '_cols_populated',
  '_columns',
  '_compile_w_cache',
  '_compiler',
  '_compiler_dispatch',
  '_constructor',
  '_copy_internals',
  '_deannotate',
  '_dialect_kwargs_traverse_internals',
  '_execute_on_con

In [87]:
user1 = user.insert().values(NAME="Hong")

In [88]:
print(user1.complie().params)

AttributeError: 'Insert' object has no attribute 'complie'